In [17]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

data_cleaned =  'https://raw.githubusercontent.com/mikdadkanbar/HousePricesML/main/data_cleaned.csv'

data_ready = 'https://raw.githubusercontent.com/mikdadkanbar/HousePricesML/main/data_ready.csv'
#data_ready is after creating the dummy variables 
df=pd.read_csv(data_ready)
 
df.head() 


,Unnamed: 0,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,60,65.0,8450,7,5,20,2003,196.0,706,...,0,0,0,0,1,0,0,0,1,0
1,1,20,80.0,9600,6,8,47,1976,0.0,978,...,0,0,0,0,1,0,0,0,1,0
2,2,60,68.0,11250,7,5,22,2002,162.0,486,...,0,0,0,0,1,0,0,0,1,0
3,3,70,60.0,9550,7,5,108,1970,0.0,216,...,0,0,0,0,1,0,0,0,0,0
4,4,60,84.0,14260,8,5,23,2000,350.0,655,...,0,0,0,0,1,0,0,0,1,0


In [14]:
#Linear Regression : 

X_train, X_test, y_train, y_test = train_test_split(df.drop('SalePrice', axis=1), df['SalePrice'], test_size=0.2, random_state=42)


lr_model = LinearRegression()

# fit the model on the training data
lr_model.fit(X_train, y_train)

y_pred = lr_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f'MSE :  {mse:.2f}')
 



